In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import openai

from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv('OPENAI_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# First experiment

In [ ]:
from utils.constants import STAGE_DIR

with open(STAGE_DIR / '01_sentences.txt', 'r') as f:
    sentences = f.readlines()

In [4]:
sentences = '''
Los trabajadores tienen como derechos básicos, con el contenido y alcance que para cada uno de los mismos disponga su específica normativa, los de Negociación colectiva

Los trabajadores menores de dieciocho años no podrán realizar trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación conforme a lo dispuesto en la Ley 31/1995, de 8 de noviembre, de Prevención de Riesgos Laborales, y en las normas reglamentarias aplicables

Si el representante legal de una persona de capacidad limitada la autoriza expresa o tácitamente para realizar un trabajo, queda esta también autorizada para ejercitar los derechos y cumplir los deberes que se derivan de su contrato y para su cesación

Podrán contratar la prestación de su trabajo Los extranjeros, de acuerdo con lo dispuesto en la legislación específica sobre la materia

El contrato de trabajo se podrá celebrar por escrito o de palabra
'''

prompt = f'''
Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to develop a spanish event extraction task. The steps to achieve it are the following:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity and an event trigger, usually in the form of a verb. It can happen that a sentence relates more than one object entity with the same subject and event trigger. The sentences can contain entities and phrases that don't correspond to any classification. Also, the object can be separated from the subject and event trigger by these complements. The definitions of each category are the following:
    * event trigger: It refers to the action that is enforced by the legal text. It can be in a negative form, e.g. "no pueden".
    * subject entity: It refers to the entity that most follow, do the action of the event trigger.
    * object entity: It refers to the entity that is the receptor of the action. In the legal domain, it can be, for example, a right, a beneficiary from the action, an institution, a non-right, etc. 
    
    Example 1:
    Input: Cuando la relación laboral sea de duración superior a cuatro semanas, el empresario deberá informar por escrito, en los términos y plazos que se establezcan reglamentariamente, al trabajador sobre los elementos esenciales del contrato y las principales condiciones de ejecución de la prestación laboral, siempre que tales elementos y condiciones no figuren en el contrato de trabajo formalizado por escrito.
    Output:
        * subject entity: "empresario"
        * object entity: "trabajador"
        * event trigger: "deberá informar por escrito"
        
    Example 2:
    Input: La denegación de la solicitud deberá ser notificada por el empresario al trabajador por escrito y de manera motivada.
    Output:
        * subject entity: "empresario"
        * object entity: "trabajador"
        * event trigger: "deberá ser notificada"

3. The output of the task should be a list of dictionaries, each dictionary contains the following keys:
    - sentence: the sentence
    - subject: the subject entity
    - object: the object entity
    - event: the event trigger
Note that it can be repeated sentences because of the different subject-event-object combinations.

The set of sentences in spanish to use is the following:

```
{sentences}
```
'''

In [6]:
response = get_completion(prompt)

'Output:\n```\n[\n    {\n        "sentence": "Los trabajadores tienen como derechos básicos, con el contenido y alcance que para cada uno de los mismos disponga su específica normativa, los de Negociación colectiva",\n        "subject": "trabajadores",\n        "object": "Negociación colectiva",\n        "event": "tienen como derechos básicos"\n    },\n    {\n        "sentence": "Los trabajadores menores de dieciocho años no podrán realizar trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación conforme a lo dispuesto en la Ley 31/1995, de 8 de noviembre, de Prevención de Riesgos Laborales, y en las normas reglamentarias aplicables",\n        "subject": "trabajadores menores de dieciocho años",\n        "object": "trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación",\n        "event": "no podrán realizar"\n    },\n    {\n        "sentenc

In [7]:
print(response)

Output:
```
[
    {
        "sentence": "Los trabajadores tienen como derechos básicos, con el contenido y alcance que para cada uno de los mismos disponga su específica normativa, los de Negociación colectiva",
        "subject": "trabajadores",
        "object": "Negociación colectiva",
        "event": "tienen como derechos básicos"
    },
    {
        "sentence": "Los trabajadores menores de dieciocho años no podrán realizar trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación conforme a lo dispuesto en la Ley 31/1995, de 8 de noviembre, de Prevención de Riesgos Laborales, y en las normas reglamentarias aplicables",
        "subject": "trabajadores menores de dieciocho años",
        "object": "trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación",
        "event": "no podrán realizar"
    },
    {
        "sentence": "Si el repres

# Experiments with `PlanTL-GOB-ES/RoBERTalex`

In [ ]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer, FillMaskPipeline
from pprint import pprint
tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model.eval()
pipeline = FillMaskPipeline(model, tokenizer_hf)
text = f"¡Hola <mask>!"
res_hf = pipeline(text)
pprint([r['token_str'] for r in res_hf])

In [ ]:
tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model.eval()
pipeline = FillMaskPipeline(model, tokenizer_hf)
text = f"El contrato de trabajo se podrá <mask>"
res_hf = pipeline(text)

pprint([r['token_str'] for r in res_hf])

In [ ]:
print(res_hf)

In [ ]:
seed = f"El contrato de trabajo se podrá <mask>"
i = 10
while i:
    tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
    model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
    model.eval()
    pipeline = FillMaskPipeline(model, tokenizer_hf)
    res_hf = pipeline(seed)
    max_score_sequence = max(res_hf, key=lambda x: x['score'])['sequence']
    seed = max_score_sequence + ' <mask>'
    i -= 1

print(seed)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/gpt2-large-bne")
model = AutoModelForCausalLM.from_pretrained("PlanTL-GOB-ES/gpt2-large-bne")
generator = pipeline('text-generation', tokenizer=tokenizer, model=model)
set_seed(42)
generator("Los  ", num_return_sequences=5)

# Zero-shot experiment

In [6]:
from utils.constants import STAGE_DIR

with open(STAGE_DIR / 'previous-work' / 'EVAL_DATA.txt') as f:
    sentences = f.read()

In [10]:
import re

sentences_re = re.split(r'\d+\t', sentences)

In [13]:
sentences_f = list(map(lambda x: x.split('\n')[0], sentences_re[1:]))

In [92]:
evaluation_data = []
for sentence in sentences_f:
    sentence_data = {}
    sentence_data['text'] = ''.join(
        re.split(r'\<\w+\>|\</\w+\>', sentence))
    subject = re.search(
        r'(?<=\<e1\>)(.+)(?=\</e1\>)', sentence
    )
    sentence_data['subject'] = subject.group(0) if subject else None
    object = re.search(
        r'(?<=\<e2\>)(.+)(?=\</e2\>)', sentence
    )
    sentence_data['object'] = object.group(0) if object else None
    event = re.search(
        r'(?<=\<rel\>)(.+)(?=\</rel\>)', sentence
    )
    sentence_data['event'] = event.group(0) if event else None
    complement = re.search(
        r'(?<=\<comp\>)(.+)(?=\</comp\>)', sentence
    )
    sentence_data['complement'] = complement.group(0) if complement else None
    evaluation_data.append(sentence_data)

In [98]:
evaluation_data

[{'text': 'Los trabajos realizados a título de amistad, benevolencia o buena vecindad.',
  'subject': 'vecindad',
  'object': 'trabajos',
  'event': 'realizados',
  'complement': None},
 {'text': 'Las situaciones de incapacidad temporal, nacimiento, adopción, guarda con fines de adopción, acogimiento, riesgo durante el embarazo, riesgo durante la lactancia y violencia de género interrumpirán el cómputo de la duración del contrato.',
  'subject': 'contrato',
  'object': None,
  'event': 'interrumpirán',
  'complement': None},
 {'text': 'Si el trabajador tuviera asignadas condiciones o retribuciones especiales en virtud de contraprestaciones establecidas en la parte no válida del contrato, el órgano de la jurisdicción social que a instancia de parte declare la nulidad hará el debido pronunciamiento sobre la subsistencia o supresión en todo o en parte de dichas condiciones o retribuciones.',
  'subject': 'órgano de la jurisdicción social',
  'object': 'trabajador',
  'event': 'hará',
  'c

In [93]:
sentences = "\n\n".join(sentence_data['text'] for sentence_data in evaluation_data)

In [173]:
prompt = f'''
We are interested in extracting event information from spanish legal text. For that, we define the following roles for an event.
- subject: Agent of the action, who performs the action.
- event trigger: Action
- object: Receiver of the action.
- complement: Item which is handled in the relation.

Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to extract the roles from the text written in Spanish following the next steps:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity, an event trigger and a complement following the previous definitions and examples.
3. The output of the task should be a list of dictionaries, each dictionary contains the following keys:
    - sentence: the sentence
    - subject: the subject entity
    - object: the object entity
    - event: the event trigger
    - complement: the complement

The set of sentences in spanish to use is the following:

```
{sentences}
```
'''

In [174]:
print(prompt)


We are interested in extracting event information from spanish legal text. For that, we define the following roles for an event.
- subject: Agent of the action, who performs the action.
- event trigger: Action
- object: Receiver of the action.
- complement: Item which is handled in the relation.

Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to extract the roles from the text written in Spanish following the next steps:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity, an event trigger and a complement following the previous definitions and examples.
3. The output of the task should be a list of dictionaries, each dictionary contains the following keys:
    - sentence: the sentence
    - subject: the subject entity
    - object: the object entity
    - event: the event trigger
    - complement: the complement

The set of sentences in span

In [175]:
response = get_completion(prompt)

In [176]:
import json

evaluation_predict = json.loads(response)

In [177]:
evaluation_predict

[{'sentence': 'Los trabajos realizados a título de amistad, benevolencia o buena vecindad.',
  'subject': 'trabajos',
  'object': None,
  'event': 'realizados',
  'complement': 'a título de amistad, benevolencia o buena vecindad'},
 {'sentence': 'Las situaciones de incapacidad temporal, nacimiento, adopción, guarda con fines de adopción, acogimiento, riesgo durante el embarazo, riesgo durante la lactancia y violencia de género interrumpirán el cómputo de la duración del contrato.',
  'subject': 'situaciones',
  'object': None,
  'event': 'interrumpirán',
  'complement': 'el cómputo de la duración del contrato'},
 {'sentence': 'Si el trabajador tuviera asignadas condiciones o retribuciones especiales en virtud de contraprestaciones establecidas en la parte no válida del contrato, el órgano de la jurisdicción social que a instancia de parte declare la nulidad hará el debido pronunciamiento sobre la subsistencia o supresión en todo o en parte de dichas condiciones o retribuciones.',
  'su

In [178]:
from utils.metrics import evaluate_label_metrics, calculate_metrics

evaluate_label_metrics(evaluation_data, evaluation_predict, 'object')

{'TP': 4, 'TN': 0, 'FP': 0, 'FN': 9}

In [138]:
calculate_metrics(evaluate_label_metrics(evaluation_data, evaluation_predict, 'object'), as_df=True) * 100

,metrics
accuracy,7.692308
precision,16.666667
recall,12.500000
f1,14.285714


In [126]:
calculate_metrics(evaluate_label_metrics(evaluation_data, evaluation_predict, 'subject'), as_df=True) * 100

,metrics
accuracy,46.153846
precision,60.000000
recall,66.666667
f1,63.157895


In [128]:
calculate_metrics(evaluate_label_metrics(evaluation_data, evaluation_predict, 'event'), as_df=True) * 100

,metrics
accuracy,76.923077
precision,76.923077
recall,100.000000
f1,86.956522


In [136]:
calculate_metrics(evaluate_label_metrics(evaluation_data, evaluation_predict, 'complement', threshold=0.1), as_df=True) * 100

,metrics
accuracy,7.692308
precision,12.500000
recall,16.666667
f1,14.285714


In [179]:
from utils.metrics import compute_average_similarity_score

print(f'''
Subject - {compute_average_similarity_score(evaluation_data, evaluation_predict, 'subject')}
Object - {compute_average_similarity_score(evaluation_data, evaluation_predict, 'object')}
Trigger - {compute_average_similarity_score(evaluation_data, evaluation_predict, 'event')}
Complement - {compute_average_similarity_score(evaluation_data, evaluation_predict, 'complement')}
''')



Subject - 0.4959087481146305
Object - 0.3076923076923077
Trigger - 0.769855144855145
Complement - 0.10340487628435256



# Few-shot experiment

In [148]:
train_data = evaluation_data[:3]
few_evaluation_data = evaluation_data[3:]
few_sentences = '\n\n'.join(s['text'] for s in few_evaluation_data)

In [145]:
subject_train_data = "\n".join(f"`{s['text']}` -> {s['subject']}" for s in train_data)
object_train_data = "\n".join(f"`{s['text']}` -> {s['object']}" for s in train_data)
event_train_data = "\n".join(f"`{s['text']}` -> {s['event']}" for s in train_data)
complement_train_data = "\n".join(f"`{s['text']}` -> {s['complement']}" for s in train_data)

In [165]:
prompt = f'''
We are interested in extracting event information from spanish legal text. For that, we define the following roles for an event. For each definition, 3 examples will be provided with the original sentece between simple backticks, an arrow and the correct role to extract.
- subject: Agent of the action, who performs the action.
{subject_train_data}
- event trigger: Action
{object_train_data}
- object: Receiver of the action.
{event_train_data}
- complement: Item which is handled in the relation.
{complement_train_data}

Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to extract the roles from the text written in Spanish following the next steps:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity, an event trigger and a complement following the previous definitions and examples.
3. The output of the task should be a list of dictionaries, each dictionary contains the following keys:
    - sentence: the sentence
    - subject: the subject entity
    - object: the object entity
    - event: the event trigger
    - complement: the complement

The set of sentences in spanish to use is the following:

```
{few_sentences}
```
'''

In [166]:
print(prompt)


We are interested in extracting event information from spanish legal text. For that, we define the following roles for an event. For each definition, 3 examples will be provided with the original sentece between simple backticks, an arrow and the correct role to extract.
- subject: Agent of the action, who performs the action.
`Los trabajos realizados a título de amistad, benevolencia o buena vecindad.` -> vecindad
`Las situaciones de incapacidad temporal, nacimiento, adopción, guarda con fines de adopción, acogimiento, riesgo durante el embarazo, riesgo durante la lactancia y violencia de género interrumpirán el cómputo de la duración del contrato.` -> contrato
`Si el trabajador tuviera asignadas condiciones o retribuciones especiales en virtud de contraprestaciones establecidas en la parte no válida del contrato, el órgano de la jurisdicción social que a instancia de parte declare la nulidad hará el debido pronunciamiento sobre la subsistencia o supresión en todo o en parte de dicha

In [167]:
few_response = get_completion(prompt)

In [168]:
print(few_response)

[
    {
        "sentence": "En caso de nulidad por discriminación salarial por razón de sexo, el trabajador tendrá derecho a la retribución correspondiente al trabajo igual o de igual valor.",
        "subject": "trabajador",
        "object": null,
        "event": "tendrá derecho",
        "complement": "retribución correspondiente al trabajo igual o de igual valor"
    },
    {
        "sentence": "Si el empresario diera un trabajo en común a un grupo de sus trabajadores, conservará respecto de cada uno, individualmente, sus derechos y deberes.",
        "subject": "empresario",
        "object": null,
        "event": "conservará",
        "complement": "respecto de cada uno, individualmente, sus derechos y deberes"
    },
    {
        "sentence": "El jefe del grupo ostentará la representación de los que lo integren, respondiendo de las obligaciones inherentes a dicha representación.",
        "subject": "jefe del grupo",
        "object": null,
        "event": "ostentará",
    

In [169]:
few_evaluation_predict = json.loads(few_response)

In [170]:
from utils.metrics import compute_average_similarity_score

print(f'''
Subject - {compute_average_similarity_score(few_evaluation_data, few_evaluation_predict, 'subject')}
Object - {compute_average_similarity_score(few_evaluation_data, few_evaluation_predict, 'object')}
Trigger - {compute_average_similarity_score(few_evaluation_data, few_evaluation_predict, 'event')}
Complement - {compute_average_similarity_score(few_evaluation_data, few_evaluation_predict, 'complement')}
''')



Subject - 0.52625
Object - 0.3
Trigger - 0.8003787878787879
Complement - 0.12466322670728074

